<h2>Fetching Data from website using API call (Air Quality Monitors -> Bristol Open Data Bristol)</h2>


In [ ]:
pip install scikit-learn

In [60]:
import numpy as np
import pandas as pd
import json
from sklearn import preprocessing

In [ ]:
import requests

url = "https://maps2.bristol.gov.uk/server2/rest/services/ext/air_quality/MapServer/0/query?outFields=*&where=1%3D1&f=geojson"

# https://www.geeksforgeeks.org/python/get-post-requests-using-python/

response = requests.get(url)

data = response.json()
# data

# Saving JSON data to a file => https://www.geeksforgeeks.org/python/reading-and-writing-json-to-a-file-in-python/
# with open('Air_Quality_Bristol.json', 'w') as file:
#     file.write(response.text)


In [ ]:
processed_data = {}

# print(data['type']) # o/p -> FeatureCollection

# print(data['features'][0]['type']) # o/p -> Feature

# print(data['features'][0]['id'])

id = data['features'][0]['id']

location = data['features'][0]["properties"]["location"]

pollutants = data['features'][0]['properties']['pollutants']

x = data['features'][150]['geometry']['coordinates'][0]
y = data['features'][150]['geometry']['coordinates'][1]

# scaler = preprocessing.MinMaxScaler()
# normalized_coordinates = scaler.fit_transform(x)

file = open('Air_Quality_Bristol.json')

file_data_in_dictionary = json.load(file)

temp_list = []

def NormalizeData(data):
    print(type(data))
    return (data - np.min(data)) / (np.max(data) - np.min(data))

for i in file_data_in_dictionary['features']:
    temp_list.append({
        'ID' : i['id'],
        'Location' : i['properties']['location'],
        'Longitude' : i['geometry']['coordinates'][1],
        'Latitude' : i['geometry']['coordinates'][0],
        'Pollutants' : i['properties']['pollutants']
    })
    
pd_dataframe = pd.DataFrame(temp_list)   

pd_dataframe['Normalized Longitude'] = (pd_dataframe['Longitude'] - pd_dataframe['Longitude'].min()) / (pd_dataframe['Longitude'].max() - pd_dataframe['Longitude'].min())
pd_dataframe['Normalized Latitude'] = (pd_dataframe['Latitude'] - pd_dataframe['Latitude'].min()) / (pd_dataframe['Latitude'].max() - pd_dataframe['Latitude'].min())

print(pd_dataframe.head())


# https://stackoverflow.com/questions/28056171/how-to-build-and-fill-pandas-dataframe-from-for-loop
# https://www.geeksforgeeks.org/python/normalize-a-column-in-pandas/

   ID           Location  Longitude  Latitude Pollutants  \
0   1   Withywood School  51.407746 -2.627749    BTX NO2   
1   2     Colston Avenue  51.454557 -2.596814        NO2   
2   3      Blackboy Hill  51.469205 -2.613994        NO2   
3   4        Three Lamps  51.444210 -2.578333        NO2   
4   5  Bedminster Parade  51.442812 -2.595294        NO2   

   Normalized Longitude  Normalized Latitude  
0              0.090933             0.450763  
1              0.293816             0.515873  
2              0.357306             0.479713  
3              0.248973             0.554769  
4              0.242914             0.519071  


In [5]:
import os
import sys

# Set absolute path for Java
os.environ["JAVA_HOME"] = os.path.abspath("./JDK-8")

# Add Java bin to PATH
os.environ["PATH"] = os.path.join(os.environ["JAVA_HOME"], "bin") + os.pathsep + os.environ["PATH"]

os.environ["PYSPARK_PYTHON"] = sys.executable
os.environ["PYSPARK_DRIVER_PYTHON"] = sys.executable

In [7]:
# Create a new Sparksession
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .appName("shushant_portfolio_spark_session") \
    .getOrCreate()